# Score

> Score functions and classes

In [ ]:
#| default_exp score

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.core import Module, Executor
from emb_opt.schemas import Item, Query, Batch, ScoreFunction, ScoreResponse

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export

class ScoreModule(Module):
    def __init__(self, function: ScoreFunction):
        super().__init__(ScoreResponse, function)
        
    def gather_inputs(self, batch: Batch) -> (List[Tuple], List[Item]):
        idxs, inputs = batch.flatten_query_results()
        return (idxs, inputs)
    
    def scatter_results(self, batch: Batch, idxs: List[Tuple], results: List[ScoreResponse]):
        for (q_idx, r_idx), result in zip(idxs, results):
            batch_item = batch.get_item(q_idx, r_idx)
            batch_item.score = result.score
            if result.data:
                batch_item.data.update(result.data)
                
            if not result.valid:
                batch_item.data['_internal']['remove'] = True
                batch_item.data['_internal']['remove_details'] = 'score response invalid'

In [ ]:
def build_batch():
    np.random.seed(42)
    d_emb = 128
    query = Query(embedding=np.random.randn(d_emb))
    query_results = [Item(embedding=np.random.randn(d_emb), data={'id':i}) for i in range(100)]
    expected_scores = [np.linalg.norm(i.embedding) for i in query_results]
    query.add_query_results(query_results)
    batch = Batch(queries=[query])
    return batch, expected_scores

def norm_score(input: Item):
    embedding = np.array(input.embedding)
    norm = np.linalg.norm(embedding)
    return ScoreResponse(valid=True, score=norm, data={'norm':norm})

def norm_score_batched(inputs: List[Item]):
    embeddings = np.array([i.embedding for i in inputs])
    norms = np.linalg.norm(embeddings, axis=-1)
    results = [ScoreResponse(valid=True, score=i, data={'norm':i}) for i in norms]
    return results

def test_score(score_module):
    batch, expected_scores = build_batch()
    batch = score_module(batch)
    scores = [i.score for i in batch[0]]
    assert np.allclose(expected_scores, scores)

func = Executor(norm_score, batched=False)
score_module = ScoreModule(func)
test_score(score_module)

score_module = ScoreModule(norm_score_batched)
test_score(score_module)

func = Executor(norm_score_batched, batched=True, batch_size=5)
score_module = ScoreModule(func)
test_score(score_module)